In [ ]:
import warnings # type: ignore
warnings.filterwarnings('ignore') # type: ignore
from bs4 import BeautifulSoup # type: ignore
import pandas as pd # type: ignore
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout # type: ignore
import time # type: ignore

GAME = 'black-white'
LOCATIONS = pd.DataFrame()

async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.webkit.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

def get_hrefs(html):
    soup = BeautifulSoup(html, 'html.parser')
    info_cards = soup.find_all('div', class_='infocard')
    href_list = []
    
    for card in info_cards:
        link = card.find('a', class_='ent-name')
        if link and link.has_attr('href'):
            href_list.append(link['href'])
    
    return href_list

html = await get_html('https://pokemondb.net/pokedex/game/black-white', '#main')
hrefs = get_hrefs(html)

for href in hrefs:
    html = await get_html(f"https://pokemondb.net{href}", '#main')
    soup = BeautifulSoup(html)
    location = pd.read_html(str(soup), attrs={'class' : 'vitals-table'})[5]
    game_row = location['Black White' in location[0]]
    game_row['pokemon'] = href.split('/')[-1]
    LOCATIONS = pd.concat([LOCATIONS, game_row], ignore_index=True)

pd.set_option('display.max_rows', 1000)
LOCATIONS